In [1]:
!nvidia-smi

Sun Oct 25 07:34:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'

checkpoint  Dockerfile	exp.ipynb   model	requirements.txt
data	    docs	main.ipynb  preprocess	service
dataloader  example	main.py     README.md	train


In [4]:
!pip install -r drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'/requirements.txt

     |████████████████████████████████| 778kB 12.3MB/s 
     |████████████████████████████████| 2.9MB 14.2MB/s 
     |████████████████████████████████| 1.1MB 22.0MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251543 sha256=6a0b842835c63ce1d3e8c6d36b2333be090aa0d8b0938ae163eb537b80314dba
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=340d36e043d23ba09957b8018be262392b418a01b25eef448f644583c3666024
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built kss sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.9.2 which is incompatible.


In [5]:
!pip install --upgrade tokenizers==0.8.1.rc1

     |████████████████████████████████| 3.0MB 9.0MB/s 
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2


In [6]:
import sys
sys.path.append('/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot')

In [7]:
import os
import numpy as np
import torch
from model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

In [8]:
root_path='/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot'
data_path = f"{root_path}/data/wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20201022-add-chatbotdata.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수



In [9]:
!nvidia-smi

Sun Oct 25 07:36:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    40W / 300W |   2857MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
while 1:
#for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
  # set top_k to 50
  sample_output = model.generate(
      input_ids=input_ids
      )


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')

# for s in kss.split_sentences(sent):
#     print(s)

Question: 안녕하세요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 안녕하세요.안녕하세요.안녕하세요.안녕하세요.안녕.안녕하세요.안녕.안녕히 주무세요.안녕히 주무세요.
----------------------------------------------------------------------------------------------------
Question: 저는 장윤혁입니다.


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 네, 맞군요.네, 더 말씀해주세요.네, 더 말씀해주세요.네, 더 말씀해주세요.네, 더 말씀해주세요.네, 더 말씀해주세요.네, 더 말씀해주세요
----------------------------------------------------------------------------------------------------
Question: 요새 힘들어요


...

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 으르렁으르렁으르렁으르렁으르렁으르렁으르렁으르렁
----------------------------------------------------------------------------------------------------
Question: 씨발놈아


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 신경쓰는 일이 많은가봐요.저도 그런 편이에요.신경쓰지 마세요.저도 그런 편이에요.의외네요.저도 그런 편이에요.의외네요.저도 그런 편
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored